In [ ]:
#!aws s3 cp s3://raw-videos-gleonato/AttackDetectionNet/data/train /home/ec2-user/SageMaker/deepfake-framework/4-AttackDetectionNet/data/ --recursive

In [43]:
# %matplotlib inline
# %config InlineBackend.figure_format = 'retina'
import matplotlib.pyplot as plt
import numpy as np
import torch
import ipywidgets
from torch import nn
from torch import optim
import torch.nn.functional as F
from torchvision import datasets, transforms, models
import boto3
from datetime import datetime
# from torch.utils.tensorboard import SummaryWriter
# writer = SummaryWriter()


In [12]:
!df -H

Filesystem      Size  Used Avail Use% Mounted on
devtmpfs         34G   87k   34G   1% /dev
tmpfs            34G   17k   34G   1% /dev/shm
/dev/nvme0n1p1  117G   99G   18G  86% /
/dev/nvme2n1    2.2T   15G  2.1T   1% /home/ec2-user/SageMaker


In [44]:
# Dataset
# data_dir = '/home/ec2-user/SageMaker/deepfake-framework/4-AttackDetectionNet/data/'
data_dir = '/home/ec2-user/SageMaker/deepfake-framework/4-AttackDetectionNet/dataset/Subset-10000/'

# S3 parms
# Bucket = 'raw-videos-gleonato'
# s3_data = 'AttackDetectionNet/data/train/'

# image input
# frame_width=1280
# frame_height=720
frame_width=320
frame_height=180
num_output_channels=3

# Model parms
epochs = 5
batch_size = 512
steps = 0
running_loss = 0
print_every = 1
train_losses, test_losses = [], []

In [8]:
train_transforms = transforms.Compose([transforms.Resize([frame_height,frame_width]),transforms.Grayscale(num_output_channels),
                                   transforms.ToTensor(),
                                   ])
test_transforms = transforms.Compose([transforms.Resize([frame_height,frame_width]),transforms.Grayscale(num_output_channels),
                                  transforms.ToTensor(),
                                  ])
train_data = datasets.ImageFolder(data_dir,       
                transform=train_transforms)
test_data = datasets.ImageFolder(data_dir,
                transform=test_transforms)
num_train = len(train_data)


In [16]:
print(train_data)
# plt.imshow(train_data[0])

# plt.imshow(train_data[0].permute(1, 2, 0))

Dataset ImageFolder
    Number of datapoints: 80212
    Root location: /home/ec2-user/SageMaker/deepfake-framework/4-AttackDetectionNet/data/
    StandardTransform
Transform: Compose(
               Resize(size=[720, 1280], interpolation=PIL.Image.BILINEAR)
               Grayscale(num_output_channels=1)
               ToTensor()
           )


In [9]:

indices = list(range(num_train))
# print(indices)

split = int(np.floor(.2 * num_train))
# print(split)

np.random.shuffle(indices)
#     print(indices)

In [12]:
from torch.utils.data.sampler import SubsetRandomSampler
train_idx, test_idx = indices[split:], indices[:split]
print(len(train_idx), len(test_idx))
train_sampler = SubsetRandomSampler(train_idx)
test_sampler = SubsetRandomSampler(test_idx)
trainloader = torch.utils.data.DataLoader(train_data,
               sampler=train_sampler, batch_size=batch_size)
testloader = torch.utils.data.DataLoader(test_data,
               sampler=test_sampler, batch_size=batch_size)

trainloader = torch.utils.data.Subset(trainloader,100)
testloader = torch.utils.data.Subset(testloader,100)

print(trainloader, testloader)
# return trainloader, testloader

64170 16042
<torch.utils.data.dataset.Subset object at 0x7fac9da0f5c0> <torch.utils.data.dataset.Subset object at 0x7fac9da0f978>


In [ ]:
train_features, train_labels = next(iter(trainloader))
print(f"Feature batch shape: {train_features.size()}")
print(f"Labels batch shape: {train_labels.size()}")
img = train_features[0].squeeze()
label = train_labels[0]
plt.imshow(img, cmap="gray")
plt.show()
print(f"Label: {label}")

In [45]:
# Carrega os datasets de treinamento 
def load_split_train_test(datadir, valid_size = .2):
    
    # Transforma resize dos frames
    train_transforms = transforms.Compose([transforms.Resize([frame_height,frame_width]),transforms.Grayscale(num_output_channels),
                                       transforms.ToTensor(),
                                       ])
    test_transforms = transforms.Compose([transforms.Resize([frame_height,frame_width]),transforms.Grayscale(num_output_channels),
                                      transforms.ToTensor(),
                                      ])
    train_data = datasets.ImageFolder(datadir,       
                    transform=train_transforms)
    test_data = datasets.ImageFolder(datadir,
                    transform=test_transforms)
    print(train_data)
    num_train = len(train_data)
    # print('Input shape:' + train_data[0][0].shape)
    indices = list(range(num_train))

    split = int(np.floor(valid_size * num_train))
    # print(split)

    np.random.shuffle(indices)
#     print(indices)

    from torch.utils.data.sampler import SubsetRandomSampler
    train_idx, test_idx = indices[split:], indices[:split]
    print(len(train_idx), len(test_idx))
    train_sampler = SubsetRandomSampler(train_idx)
    test_sampler = SubsetRandomSampler(test_idx)
    trainloader = torch.utils.data.DataLoader(train_data,
                   sampler=train_sampler, batch_size=batch_size)
    testloader = torch.utils.data.DataLoader(test_data,
                   sampler=test_sampler, batch_size=batch_size)
    return trainloader, testloader

trainloader, testloader = load_split_train_test(data_dir, .2)
print(trainloader.dataset.classes)
# print(testloader.dataset)

# Testa para GPU e Carrega Resnet50 

device = torch.device("cuda:0,1,2,3" if torch.cuda.is_available() 
                                  else "cpu")
print(device)
model = models.resnet50(pretrained=True)


#  freeze Resnet50 pre-trained layers, so we don’t backprop through them during training. 
for param in model.parameters():
    param.requires_grad = False

# Then, we re-define the final fully-connected the layer, the one that we’ll train with our images. We also create the criterion (the loss function) and pick an optimizer (Adam in this case) and learning rate.

model.fc = nn.Sequential(nn.Linear(2048, 512),
                                 nn.ReLU(),
                                 nn.Dropout(0.2),
                                 nn.Linear(512, 10),
                                 nn.LogSoftmax(dim=1))
criterion = nn.NLLLoss()
optimizer = optim.Adam(model.fc.parameters(), lr=0.003)
model = nn.DataParallel(model)
print(model)
model.to(device)



# Train the model
for epoch in range(epochs):
    for inputs, labels in trainloader:
        steps += 1
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        logps = model.forward(inputs)
        loss = criterion(logps, labels)
#         writer.add_scalar("Loss/train", loss, epoch)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        print(loss)
        
        if steps % print_every == 0:
            test_loss = 0
            accuracy = 0
            model.eval()
            with torch.no_grad():
                for inputs, labels in testloader:
                    inputs, labels = inputs.to(device), labels.to(device)
                    logps = model.forward(inputs)
                    batch_loss = criterion(logps, labels)
#                     writer.add_scalar("Loss/train", batch_loss, epoch)
                    test_loss += batch_loss.item()
                    
                    ps = torch.exp(logps)
                    top_p, top_class = ps.topk(1, dim=1)
                    equals = top_class == labels.view(*top_class.shape)
                    accuracy += torch.mean(equals.type(torch.FloatTensor)).item()
                    print('Epoch: {} Step: {}'.format(epoch+1,steps))
                    print('batch loss: {}'.format(batch_loss))
                    print('Accuracy: {}'.format(accuracy/len(testloader)))
            train_losses.append(running_loss/len(trainloader))
            test_losses.append(test_loss/len(testloader))                    
            print(f"Epoch {epoch+1}/{epochs}.. "
                  f"Train loss: {running_loss/print_every:.3f}.. "
                  f"Test loss: {test_loss/len(testloader):.3f}.. "
                  f"Test accuracy: {accuracy/len(testloader):.3f}")
            running_loss = 0
            model.train()
#             writer.flush()
            now = datetime.now()
            dt_string = now.strftime("%d-%m-%Y-%H:%M:%S")
    torch.save(model, 'AttackDetectionNetModel-epoch-{}-steps-{}.pth'.format(epoch+1,dt_string))

Dataset ImageFolder
    Number of datapoints: 20000
    Root location: /home/ec2-user/SageMaker/deepfake-framework/4-AttackDetectionNet/dataset/Subset-10000/
    StandardTransform
Transform: Compose(
               Resize(size=[180, 320], interpolation=PIL.Image.BILINEAR)
               Grayscale(num_output_channels=3)
               ToTensor()
           )
16000 4000
['Attacked', 'NotAttacked']
cuda:0
DataParallel(
  (module): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, ker

KeyboardInterrupt: 